# CRITIC - TOPSIS

Bu çalışmada Çok Kriterli Karar Verme Teknikleri (Multi Criteria Decision Making) kullanılarak bir performans sıralaması yapılacaktır. Kullanılan veri seti  R-ın içerisinde bulunan "mtcars" isimli veri seti olacaktır. Bu datasetde bulunan veriler 1974 _Motor Trend US_ dergisinden alınmıştır ve 32 otomobil (1973–74 modelleri) için 10 kriterli bir veri tablosudur. Kullanıcağımız sıralama tekniği MCDM metotları içersinde sık bir kullanıma sahip olan TOPSİS (Technique for Order Preference by Similarity) metodu olacaktır. Kriterlerin ağırlıklandırılması için herhangi bir subjektiv değerlendirme ihtiyacı duymayan objektiv ağırlakdırma metotlarından CRİTİC (Criteria Importance Through Intercriteria Correlation) metodu kullanılacaktır. CRITIC metodu  isminden de anlaşılacağı gibi ağırlıklandırmayı kriterlerin korelasyon katsayılarına göre yapar.

Öncelikle veri setimizi tanıyalım ve hangi değişkenlerin maliyet hangilerinin kazanc olduğuna karar verelim. Az önce belirtdiğim gibi veri setimiz 32 araba ve 10 kriterden oluşuyor. Amacımız bu kriterlere göre bir performans sıralaması yapmaktır. Veri setimizde yer alan kriterler şu şekildedir:

mpg (Miles/(US) gallon) - Galon başına gidilecek yol (Kazanc)

cyl (Number of cylinders) - Silindir sayısı (Kazanc)

disp (Displacement (cu.in.) - Motor hacmi (Kazanc)

hp (Gross horsepower) - Beygir gücü (Kazanc)

drat (Rear axle ratio) - Kısaca çekme kapasitesi. Ama bu oranın fazla olması yakıt tüketimini 

artıracağından bu kriteri ben maliyet kriteri olarak değerlendireceğim. (Maliyet)

wt (Weight (1000 lbs)) - Ağırlık.  Ağırlığın fazla olması yakıt tüketimini artırır (Maliyet)

qsec (1/4 mile time) - Hızlanma ölçüsü. (Kazanc) 

am (Transmission Type) - Aracın otomotik veya manuel şanzıman konfigurasyonuna sahip olduğunu 

bildiren değişken. Bu ve bundan sonraki değişkenleri analizimde kullanmayacağım.

gear (Number of forward gears) - Vites sayısı

carb (Number of carburetors) - Karbürator sayısı

In [1]:

df = mtcars
head(df)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


Şimdi veri setimde istemediğim vs, am, gear ve carb değişkenlerini çıkartalım  

In [2]:
df = df[,1:7]
topsis_df = df[,1:7]

İlk önce CRİTİC metodu ile kriterlerimizi normalize edicem. Bunun için 2 tane fonksiyon oluşturmam gerek. Maliyet kriterlerine ayrı Kazanca kriterlerine ayrı bir formul uygulanacaktır. 

In [3]:
func_kazanc = function(x){
  (max(x) - x)/(max(x)-min(x))
}


func_kayip = function(x){
  (x - min(x))/(max(x) - min(x))
  
}


Şimdi karar matrisim olan df veri setimi oluşturduğum fonksiyonlar yardımı ile normalize ediyorum

In [4]:
df$mpg = func_kazanc(df$mpg)
df$cyl = func_kazanc(df$cyl)
df$disp = func_kazanc(df$disp)
df$hp = func_kazanc(df$hp)
df$drat = func_kayip(df$drat)
df$wt = func_kayip(df$wt)
df$qsec = func_kazanc(df$qsec)

head(df)

,mpg,cyl,disp,hp,drat,wt,qsec
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,0.5489362,0.5,0.7782489,0.7950530,0.5253456,0.2830478,0.7666667
Mazda RX4 Wag,0.5489362,0.5,0.7782489,0.7950530,0.5253456,0.3482485,0.7000000
Datsun 710,0.4723404,1.0,0.9079571,0.8551237,0.5023041,0.2063411,0.5107143
Hornet 4 Drive,0.5319149,0.5,0.5337990,0.7950530,0.1474654,0.4351828,0.4119048
Hornet Sportabout,0.6468085,0.0,0.2793714,0.5653710,0.1797235,0.4927129,0.7000000
Valiant,0.6723404,0.5,0.6161137,0.8127208,0.0000000,0.4978266,0.3190476


Yukardaki çıktı benim karar matrisimin normalize durumudur. İkinci adımımız kriterlerin corelasyon matrisini bulmak olacaktır. 

In [5]:
df_cor = cor(df)
df_cor

,mpg,cyl,disp,hp,drat,wt,qsec
mpg,1.0000000,-0.8521620,-0.8475514,-0.7761684,-0.68117191,0.8676594,0.41868403
cyl,-0.8521620,1.0000000,0.9020329,0.8324475,0.69993811,-0.7824958,-0.59124207
disp,-0.8475514,0.9020329,1.0000000,0.7909486,0.71021393,-0.8879799,-0.43369788
hp,-0.7761684,0.8324475,0.7909486,1.0000000,0.44875912,-0.6587479,-0.70822339
drat,-0.6811719,0.6999381,0.7102139,0.4487591,1.00000000,-0.7124406,-0.09120476
wt,0.8676594,-0.7824958,-0.8879799,-0.6587479,-0.71244065,1.0000000,0.17471588
qsec,0.4186840,-0.5912421,-0.4336979,-0.7082234,-0.09120476,0.1747159,1.00000000


Şimdi 1den elde ettiğimiz korelasyon matrisini çıkıp bir data frame formatına dönüştürüyoruz. 

In [6]:
df_new = as.data.frame(1-df_cor)
df_new

,mpg,cyl,disp,hp,drat,wt,qsec
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
mpg,0.0000000,1.85216196,1.84755138,1.7761684,1.6811719,0.1323406,0.5813160
cyl,1.8521620,0.00000000,0.09796713,0.1675525,0.3000619,1.7824958,1.5912421
disp,1.8475514,0.09796713,0.00000000,0.2090514,0.2897861,1.8879799,1.4336979
hp,1.7761684,0.16755255,0.20905141,0.0000000,0.5512409,1.6587479,1.7082234
drat,1.6811719,0.30006189,0.28978607,0.5512409,0.0000000,1.7124406,1.0912048
wt,0.1323406,1.78249579,1.88797992,1.6587479,1.7124406,0.0000000,0.8252841
qsec,0.5813160,1.59124207,1.43369788,1.7082234,1.0912048,0.8252841,0.0000000


Şimdi her bir kriterin yukarda oluşturduğumuz değerlerini toplayıp normalize karar matrisindeki standart sapması ile çarparak her kriter için tek bir değer elde ediyoruz

In [7]:
mpg = sum(df_new$mpg)*sd(df$mpg)
cyl = sum(df_new$cyl)*sd(df$cyl)
disp = sum(df_new$disp)*sd(df$disp)
hp = sum(df_new$hp)*sd(df$hp)
drat = sum(df_new$drat)*sd(df$hp)
wt = sum(df_new$wt)*sd(df$wt)
qsec = sum(df_new$qsec)*sd(df$qsec)

Ve son olarak elde ettiğimiz bu değerleri 0 ve 1 aralığına dönüştürmek için her bir kriter değerini kriterlerin toplam değerine bölelim

In [8]:
C_toplam = sum(c(mpg, cyl, disp, hp, drat, wt, qsec))

mpgW = mpg/C_toplam
cylW = cyl/C_toplam
dispW = disp/C_toplam
hpW = hp/C_toplam
dratW = drat/C_toplam
wtW = wt/C_toplam
qsecW = qsec/C_toplam


Agirliklar = data.frame(Kriterler = c("mpg", "cyl", "disp", "hp", "drat", "wt", "qsec"),
                 Agirliklar = c(c(mpgW, cylW, dispW, hpW, dratW, wtW, qsecW))
                 )
Agirliklar


Kriterler,Agirliklar
<chr>,<dbl>
mpg,0.1581916
cyl,0.2026432
disp,0.1396973
hp,0.1152661
drat,0.1068157
wt,0.1568359
qsec,0.1205502


Görüldüğü gibi  CRİTİC yöntemine göre en önemli kriter cyl yani silindir sayısıdır. Tüm ağırlıkların toplamı ise 1-dir. Şimdi bu kriter ve ağırlıklar yardımı ile arabaların sıralamasını TOPSİS yöntemi kullanarak bulalım. Bunun için R-nin içinde bulunan MCDM kütüphanesini kullanacağım. 

In [9]:
install.packages("MCDM")
library(MCDM)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘RankAggreg’




In [10]:
Karar_matrisi = as.matrix(topsis_df)
Kriter_Agirliklar = Agirliklar$Agirliklar
MinMax = c("max","max","max","max","min","min","max")

In [12]:
Siralama = TOPSISVector(decision = Karar_matrisi, weights = Kriter_Agirliklar, cb = MinMax)
Siralama

Alternatives,R,Ranking
<int>,<dbl>,<int>
1,0.4008507,21
2,0.3881396,25
3,0.3713571,26
4,0.4434383,16
5,0.5578325,5
6,0.3916529,24
7,0.5652268,3
8,0.3435104,32
9,0.3506607,29


Son çıktıda alternatıvlerin yani arabaların ismi değilde ID numaraları geldi. Araba isimlerinide veriye aktararak daha iyi bir çıktı oluşturalım. 

In [13]:
Siralama$Alternatives = rownames(df)
Siralama

Alternatives,R,Ranking
<chr>,<dbl>,<int>
Mazda RX4,0.4008507,21
Mazda RX4 Wag,0.3881396,25
Datsun 710,0.3713571,26
Hornet 4 Drive,0.4434383,16
Hornet Sportabout,0.5578325,5
Valiant,0.3916529,24
Duster 360,0.5652268,3
Merc 240D,0.3435104,32
Merc 230,0.3506607,29


Bu çalışmada R-nin kendi içinde bulunan _mtcars_ isimli veri seti CRITIC ve TOPSIS yöntemlerine göre incelenmiştir. Kriterlerin ağırlıklandırılması CRİTİC yöntemiyle ve sıralanması ise TOPSIS yöntemi ile gerçekleştirilmiştir. Sonuç olarak en iyi performans skorlu araba 0,588-lik skor ile Ford Pantera L seçilmiştir. İkinci Maserati Bora ise 0,04 gibi çok küçük farkla 0,584 skorunu almıştır. Duster ise 0,56 skor ile 3 cü olmuştur. 

Okuduğunuz için teşekkürler